In [1]:
import pandas as pd
import pymysql
from pymysql import Error

In [2]:
def create_connection(host,user,password,database,port):
    connection = None
    try:
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )
        return(connection)
    except Error as e:
        print(f"The error '{e}' occurred")

In [3]:
create_connection('localhost','root','Sofia2010','w3schools',3306)

#### Вопрос 1

Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.


In [4]:
con = create_connection('localhost','root','Sofia2010','w3schools',3306)
                      
query = """
SELECT CustomerName,
       Country,
       City,
       Address
FROM Customers
WHERE Country = 'Germany'
  OR Country = 'France'
  OR City = 'Madrid'
ORDER BY Country,
         CustomerName
         """

with con:
 
    cur = con.cursor()
    cur.execute(query=query)
 
    rows = cur.fetchall()
    data_query = []
    for row in rows:
        data_query.append(row)

data = pd.DataFrame(data_query, columns=['CustomerName', 'Country', 'City', 'Address'])
data        

,CustomerName,Country,City,Address
0,Blondel père et fils,France,Strasbourg,"24, place Kléber"
1,Bon app',France,Marseille,"12, rue des Bouchers"
2,Du monde entier,France,Nantes,"67, rue des Cinquante Otages"
3,Folies gourmandes,France,Lille,"184, chaussée de Tournai"
4,France restauration,France,Nantes,"54, rue Royale"
5,La corne d'abondance,France,Versailles,"67, avenue de l'Europe"
6,La maison d'Asie,France,Toulouse,1 rue Alsace-Lorraine
7,Paris spécialités,France,Paris,"265, boulevard Charonne"
8,Spécialités du monde,France,Paris,"25, rue Lauriston"
9,Victuailles en stock,France,Lyon,"2, rue du Commerce"


#### Вопрос 2

Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [5]:
con = create_connection('localhost','root','Sofia2010','w3schools',3306)
                      
query = """
SELECT Country,
       COUNT(*)
FROM Customers
GROUP BY Country
ORDER BY COUNT(*) DESC
LIMIT 3
        """

with con:
 
    cur = con.cursor()
    cur.execute(query=query)
 
    rows = cur.fetchall()
    data_query = []
    for row in rows:
        data_query.append(row)
    # print(rows)

data = pd.DataFrame(data_query, columns=['Country', 'Count_Customer'])
data        

,Country,Count_Customer
0,USA,13
1,Germany,11
2,France,11


#### Вопрос 3

Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [6]:
con = create_connection('localhost','root','Sofia2010','w3schools',3306)
                      
query = """
SELECT s.ShipperName,
       o.OrderDate
FROM
  (SELECT *
   FROM
     (SELECT *
      FROM orders
      ORDER BY OrderDate ASC
      LIMIT 10) a
   ORDER BY OrderDate
   LIMIT 1) o,
     shippers s
WHERE o.ShipperID = s.ShipperID
        """

with con:
 
    cur = con.cursor()
    cur.execute(query=query)
 
    rows = cur.fetchall()
    data_query = []
    for row in rows:
        data_query.append(row)

data = pd.DataFrame(data_query, columns=['ShipperName', 'OrderDate'])
data   

,ShipperName,OrderDate
0,Federal Shipping,1996-07-04


#### Вопрос 4.

Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [7]:
con = create_connection('localhost','root','Sofia2010','w3schools',3306)

query = """
SELECT od.OrderId,
       p.ProductName,
       p.Unit,
       od.Quantity,
       p.Price,
       p.Price*od.Quantity
FROM
  (SELECT o.OrderID
   FROM Orders o,
        order_details od,
        Products p
   WHERE o.OrderID = od.OrderID
     AND od.ProductID = p.ProductID
   GROUP BY o.OrderID
   ORDER BY sum(od.Quantity*p.Price) DESC LIMIT 1) h,
     Orders o,
     order_details od,
     Products p
WHERE o.OrderID = h.OrderID
  AND od.OrderID = h.OrderID
  AND od.ProductID = p.ProductID
"""

with con:
 
    cur = con.cursor()
    cur.execute(query=query)
 
    rows = cur.fetchall()
    data_query = []
    for row in rows:
        data_query.append(row)

data = pd.DataFrame(data_query, columns=['OrderId','ProductName','Unit','Quantity','Price','Cost'])
data   

,OrderId,ProductName,Unit,Quantity,Price,Cost
0,10372,Sir Rodney's Marmalade,30 gift boxes,12,81.0,972.0
1,10372,Côte de Blaye,12 - 75 cl bottles,40,263.5,10540.0
2,10372,Camembert Pierrot,15 - 300 g rounds,70,34.0,2380.0
3,10372,Mozzarella di Giovanni,24 - 200 g pkgs.,42,34.8,1461.6


#### Вопрос 5

Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.


In [8]:
con = create_connection('localhost','root','Sofia2010','w3schools',3306)
                      
query = """
SELECT p.ProductName,
       od.OrderID,
       od.Quantity
FROM order_details od
INNER JOIN products p ON od.ProductID = p.ProductID
WHERE od.ProductID =
    (SELECT ProductID
     FROM order_details od
     GROUP BY ProductID
     ORDER BY SUM(Quantity) DESC
     LIMIT 1)
ORDER BY od.Quantity DESC
        """

with con:
 
    cur = con.cursor()
    cur.execute(query=query)
 
    rows = cur.fetchall()
    data_query = []
    for row in rows:
        data_query.append(row)

data = pd.DataFrame(data_query, columns=['ProductName','OrderID','Quantity'])
data

,ProductName,OrderID,Quantity
0,Gorgonzola Telino,10359,70
1,Gorgonzola Telino,10390,60
2,Gorgonzola Telino,10342,56
3,Gorgonzola Telino,10376,42
4,Gorgonzola Telino,10272,40
5,Gorgonzola Telino,10393,32
6,Gorgonzola Telino,10356,30
7,Gorgonzola Telino,10374,30
8,Gorgonzola Telino,10335,25
9,Gorgonzola Telino,10253,20


#### Вопрос 6

Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.


In [9]:
con = create_connection('localhost','root','Sofia2010','w3schools',3306)
                      
query = """
SELECT SupplierName,
       Country,
       ContactName,
       Phone
FROM
  (SELECT p.SupplierID,
          COUNT(od.OrderID)
   FROM order_details od
   INNER JOIN products p ON od.ProductID = p.ProductID
   GROUP BY p.SupplierID
   ORDER BY COUNT(od.OrderID) DESC
   LIMIT 5) a
INNER JOIN suppliers s ON a.SupplierID = s.SupplierID
        """

with con:
 
    cur = con.cursor()
    cur.execute(query=query)
 
    rows = cur.fetchall()
    data_query = []
    for row in rows:
        data_query.append(row)

data = pd.DataFrame(data_query, columns=['SupplierName', 'Country', 'ContactName','Phone'])
data   

,SupplierName,Country,ContactName,Phone
0,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
1,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323


#### Вопрос 7

Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.


In [10]:
con = create_connection('localhost','root','Sofia2010','w3schools',3306)
                      
query = """
SELECT cust.Country,
       cat.CategoryName,
       sum(od.Quantity*p.Price) TotalPrice
FROM Orders o,
     Customers cust,
     order_details od,
     Products p,
     Categories cat
WHERE o.CustomerID = cust.CustomerID
  AND cust.Country = 'Brazil'
  AND o.OrderID = od.OrderID
  AND od.ProductID = p.ProductID
  AND p.CategoryID = cat.CategoryID
GROUP BY p.CategoryID
ORDER BY sum(od.Quantity*p.Price) DESC
LIMIT 1
        """

with con:
 
    cur = con.cursor()
    cur.execute(query=query)
 
    rows = cur.fetchall()
    data_query = []
    for row in rows:
        data_query.append(row)

data = pd.DataFrame(data_query, columns=['Country', 'CategoryName','TotalPrice'])
data 

,Country,CategoryName,TotalPrice
0,Brazil,Beverages,13690.0


#### Вопрос 8

Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [11]:
con = create_connection('localhost','root','Sofia2010','w3schools',3306)
                      
query = """
SELECT (max(orderPrices.OrderPrice) - min(orderPrices.OrderPrice)) PriceDifference
FROM
  (SELECT o.OrderID,
          sum(od.Quantity*p.Price) OrderPrice
   FROM Orders o,
        Order_Details od,
        Products p,
        Customers cust
   WHERE o.OrderID = od.OrderID
     AND p.ProductID = od.ProductID
     AND cust.CustomerID = o.CustomerID
     AND cust.Country = 'USA'
   GROUP BY o.OrderID) orderPrices
        """

with con:
 
    cur = con.cursor()
    cur.execute(query=query)
 
    rows = cur.fetchall()
    print('Разница в стоимости между самым дорогим и самым дешевым заказом из США равна', rows[0][0])

Разница в стоимости между самым дорогим и самым дешевым заказом из США равна 7638.45


#### Вопрос 9

Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [12]:
con = create_connection('localhost','root','Sofia2010','w3schools',3306)
                      
query = """
SELECT COUNT(OrderID),
	   CONCAT(e.FirstName,e.LastName)
FROM Orders o,
     Employees e
WHERE o.EmployeeID = e.EmployeeID
GROUP BY e.EmployeeID
ORDER BY e.BirthDate DESC
LIMIT 3
        """

with con:
 
    cur = con.cursor()
    cur.execute(query=query)
 
    rows = cur.fetchall()
    data_query = []
    for row in rows:
        data_query.append(row)

data = pd.DataFrame(data_query, columns=['OrdersCount','FullName'])
data   

,OrdersCount,FullName
0,6,AnneDodsworth
1,29,NancyDavolio
2,31,JanetLeverling


#### Вопрос 10
Сколько банок крабового мяса всего было заказано.

In [13]:
con = create_connection('localhost','root','Sofia2010','w3schools',3306)
                      
query = """
SELECT SUM(Quantity)
FROM order_details od
INNER JOIN products p ON od.ProductID = p.ProductID
WHERE p.ProductName = 'Boston Crab Meat'
        """

with con:
 
    cur = con.cursor()
    cur.execute(query=query)
 
    rows = cur.fetchall()
    print('Всего было заказано',rows[0][0],'банок крабового мяса')

Всего было заказано 256 банок крабового мяса
